In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline

## モデルの保存と読み込み

In [6]:
from statistics import mean

net = nn.Sequential(
    nn.Linear(2, 5),
    nn.ReLU(),
    nn.Linear(5, 5),
    nn.ReLU(),
    nn.Linear(5, 3)
)

x = torch.Tensor([[1, 2], [3, 4], [5, 6]])
y = torch.tensor([2, 1, 0])
opt = optim.Adam(net.parameters())
loss_f = nn.CrossEntropyLoss()
losses = []

for epoch in range(500):
    ypred = net(x)
    loss = loss_f(ypred, y)
    net.zero_grad()
    loss.backward()
    opt.step()
    losses.append(loss.item())
print(mean(losses))

0.6638195152282715


In [9]:
params = net.state_dict()
torch.save(params, "./data/net.prm", pickle_protocol=4)

In [10]:
params = torch.load("./data/net.prm", map_location="cpu")
net.load_state_dict(params)

## ONNXを使用した他のフレームワークとの連携

In [11]:
from torchvision import models

def create_network():
    net = models.resnet18()
    fc_input_dim = net.fc.in_features
    net.fc = nn.Linear(fc_input_dim, 2)
    return net

net = create_network()
prm = torch.load("./data/taco_burrito.prm", map_location="cpu")
net.load_state_dict(prm)
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [12]:
from torch import onnx

dummy_data = torch.empty(1, 3, 224, 224, dtype=torch.float32)
onnx.export(net, dummy_data, "tacoburrito.onnx")

In [17]:
import onnx
from caffe2.python.onnx import backend as caffe2_backend

onnx_model = onnx.load("taco_burrito.onnx")
backend = caffe2_backend.prepare(onnx_model)

CRITICAL:root:Cannot load caffe2.python. Error: /home/yochimonji/miniconda3/envs/pytorch_v04/lib/python3.6/site-packages/caffe2/python/caffe2_pybind11_state.cpython-36m-x86_64-linux-gnu.so: undefined symbol: _ZTIN6caffe27NetBaseE
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/yochimonji/miniconda3/envs/pytorch_v04/lib/python3.6/site-packages/caffe2/python/_import_c_extension.py", line 16, in <module>
    from caffe2.python.caffe2_pybind11_state_gpu import *  # noqa
ModuleNotFoundError: No module named 'caffe2.python.caffe2_pybind11_state_gpu'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yochimonji/miniconda3/envs/pytorch_v04/lib/python3.6/site-packages/caffe2/python/_import_c_extension.py", line 22, in <module>
    from caffe2.python.caffe2_pybind11_state_hip import *  # noqa
ModuleNotFoundError: No module named 'caffe2.python.caffe2_pybind11_state_hip'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yochimonji/miniconda3/envs/pytorch_v04/lib/python3.6/site-packages/caffe2/python/_import_c_extension.py", line 33, in <module>
    from caffe2.python.caffe2_pybind11_sta

TypeError: must be str, not list